In [3]:
from IPython.core.display import display
from gremlin_python import statics
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.process.graph_traversal import __
from gremlin_python.structure.graph import Graph, Vertex, Edge, Element
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import Cardinality
from gremlin_python.process.traversal import Column
from gremlin_python.process.traversal import Direction
from gremlin_python.process.traversal import Operator
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Pop
from gremlin_python.process.traversal import Scope
from gremlin_python.process.traversal import Barrier
from gremlin_python.process.traversal import Bindings
from gremlin_python.process.traversal import WithOptions

from functools import reduce
from pprint import pprint
import rdflib
import pathlib
import logging

import rdf2g

In [4]:
g = rdf2g.setup_graph() 

2019-10-19 16:23:10,034 [INFO] Successfully connected to the graph server


In [3]:
# OUTPUT_FILE_LAM_PROPERTIES = pathlib.Path("../output/lam_project_properties_v2.ttl").resolve()
# gg = rdflib.Graph()
# gg.parse(str(OUTPUT_FILE_LAM_PROPERTIES), format="ttl")
# print("dfdf")

# for s, p, o in gg:
#     # find s in the property graph
#     #     existentURIlist = g.V().has("iri",str(s)).id().toList()
#     #     if existentURIlist:
#     #         v1 = g.V(existentURIlist[0])
#     #
#     # if o is literal add it as value to key p
#     # else create node o and add propoerty to o
#     v1 = create_node(g,s,gg)
#     v1 = create_node(g,o,gg)
#     #v1 = g.addV(s).property('iri',str(s) ).next()
#     #v2 = g.addV(o).property('iri',str(o) ).next()
#     #     g.V(Bindings.of('id',v1)).addE(p).to(v2).iterate()

In [98]:
# g.V().drop().iterate()


# s = g.V().has('iri','http://publications.europa.eu/resources/authority/celex/md_CODE').union(__.valueMap(), __.out().valueMap().tree()).toList()
# pprint(s)

# s = g.V().has('iri','http://publications.europa.eu/resources/authority/celex/md_CODE').outE().inV().tree().next()

# pprint(s)
# pprint( s["@value"][0]["value"]["@value"][0]["key"].label  )
# ss = g.V(s["@value"][0]["key"]).propertyMap().next()  
# print (ss["dct:created"][0].value)


# {'@type': 'g:Tree',
#  '@value': [{'key': v[309],
#              'value': {'@type': 'g:Tree',
#                        '@value': [{'key': v[337],
#                                    'value': {'@type': 'g:Tree', '@value': []}},
#                                   {'key': v[386],
#                                    'value': {'@type': 'g:Tree', '@value': []}},
#                                   {'key': v[349],
#                                    'value': {'@type': 'g:Tree',
#                                              '@value': []}}]}}]}


# def flatten_list_of_dicts(list_of_dicts):
#     """
#         Flatten the list of dictionaries:
#         This is an expression for Python 3.5 or greater that merges dictionaries using reduce
#         Warning: repeating kets will be overwitten.
#     """
#     return reduce(lambda x, y: {**x, **y}, list_of_dicts, {})


# def node_properties(node, g):
#     """
#         return the node propoerties as simple dictionary
#     """
#     props = g.V(node).propertyMap().next()
#     # reduction of props to simple dict
#     props = {key: [e.value for e in props[key]] for key in props }
#     # deflating the values from lists to simple values if there is a single element provided
#     return {key: props[key] if len(props[key])>1 else props[key][0] for key in props}
    
    
# def expand_tree(tree_dict, g):
#     """
#         Given a tree with a single root node, return the simple dict corresponding to the tree
#         If multiple roots are provided, they are treated as individual tree and the result is a list of dict trees

#         The tree must be specified in a sequence of node-edge-node. Otherwise the behaviour is not predictable.

#         expecting a gremlin tree of vertices/edges, for example:
#         g.V(233).outE().inV().tree().next()
#     """
#     logging.debug('Expanding the tree and casting it into plain dict')
#     if "@type" in tree_dict and "@value" in tree_dict:
#         if not tree_dict["@value"]:
#             # reached a leaf
#             return None
#         else:
#             # iterate the tree nodes on 'key' and their children on 'value'
#             expected_list_of_dicts = [expand_tree(kv, g) for kv in tree_dict["@value"]]

#             # prime if the list of dicts are edges or nodes, nodes have '@id' and '@label' property while edges don't
#             if expected_list_of_dicts and "@id" in expected_list_of_dicts[0]:
#                 # we have a list of node descriptions
#                 return expected_list_of_dicts
#             else:
#                 # we have a list od edges, which need to be flattened because in the upper step they
#                 # are merged with the data properties of the node
#                 return flatten_list_of_dicts(expected_list_of_dicts)

#     elif "key" in tree_dict and "value" in tree_dict:
#         # process edge
#         if isinstance(tree_dict["key"], Edge):
#             return {tree_dict["key"].label: expand_tree(tree_dict["value"], g)}
#         # process node (expand the node properties and merge with edges)
#         elif isinstance(tree_dict["key"], Vertex):
#             #             props = {k:(v[0].value if v[0] else None) for k,v in props}
#             props = get_node_properties(g, tree_dict["key"])
#             props["@id"] = tree_dict["key"].id
#             props["@label"] = tree_dict["key"].label

#             expected_dict_of_edge_props = expand_tree(tree_dict["value"], g)
#             if expected_dict_of_edge_props:
#                 # deflating the values from lists to simple values if there is a single element provided
#                 expected_dict_of_edge_props = {
#                     key: expected_dict_of_edge_props[key] if len(expected_dict_of_edge_props[key]) > 1 else
#                     expected_dict_of_edge_props[key][0] for key in expected_dict_of_edge_props}
#                 # merge the two dictionaries, second overwriting the existing keys of the first one (Python3.5+)
#                 return {**props, **expected_dict_of_edge_props}
#             else:
#                 return props
            
                        
# TODO: ADD to the library

def nodes_by_type_label(g, label):
    """
        return nodes with a rdf type @label
    """
    return g.V().as_("node").where(__.out("rdf:type").hasLabel(label)).select("node").dedup().toList()    

def nodes_by_type_iri(g,iri):
    """
        return nodes with a rdf type @iri
    """
    return g.V().as_("node").where(__.out("rdf:type").properties("iri").hasValue(iri)).select("node").dedup().toList()

def node_traversal_tree(g, node, max_depth=3):
    """
        retreive the traversal tree with depth @depth for the @node
    """
    return g.V(node).repeat(__.outE().inV().dedup()).until(__.outE().count().is_(0).or_().loops().is_(max_depth)).dedup().tree().next()

In [108]:
# s = g.V().has('iri','http://publications.europa.eu/resources/authority/lam/md_CODE').outE().inV().tree().next()
# s = g.V().outE().inV().tree().next()
# s = g.V().as_("a").where(__.outE("rdf:type").inV("skos:ConceptScheme")).select("a").label().toList()

# s = g.V().as_("concept_scheme").where(__.out("rdf:type").hasLabel("skos:ConceptScheme")).select("concept_scheme").label().toList()
#
# s = g.V().as_("node").where(__.out("rdf:type").hasLabel("skos:Concept")).select("node").repeat(__.outE().inV().dedup()).times(1).label().toList()
# s = g.V().as_("node").where(__.out("rdf:type").properties("iri").hasValue("http://www.w3.org/2004/02/skos/core#ConceptScheme")).select("node").dedup().toList()

# pprint(get_nodes_by_type_label(g,"skos:ConceptScheme"))
# pprint(get_nodes_by_type_label(g,"skos:Concept"))

# pprint (get_nodes_by_type_iri(g, "http://www.w3.org/2004/02/skos/core#Cept") )

iri1 = "http://publications.europa.eu/resources/authority/celex/md_CODE"
iri2 = "http://www.w3.org/2004/02/skos/core#Concept"

source_iri = iri1
target_iri = iri2

# two known nodes
# edges = g.V().has("iri", str(source_iri)).outE().as_("q").inV().has("iri", str(target_iri)).select("q").toList()

# target unspecified
## edges = g.V().has("iri", str(source_iri)).outE().as_("q").inV().select("q").toList()
edges = g.V().has("iri", str(source_iri)).outE().toList()

# source unspecified
## edges = g.V().outE().as_("q").inV().has("iri", str(target_iri)).select("q").toList()
# edges = g.V().has("iri", str(target_iri)).inE().toList()



pprint ( g.V().hasLabel("celexd:md_DTN").next())

cs = get_nodes_by_type_label(g,"skos:Concept")

# pprint (cs)

# tr = get_node_traversal_tree(g, g.V(864).next(), 1)
tr = g.V(864).repeat(__.outE().inV().dedup()).until(__.outE().count().is_(0).or_().loops().is_(1)).dedup().tree().next()

pprint(tr)

# # pprint(s)
j = rdf2g.expand_tree(tr, g)
pprint(j)

v[864]
{'@type': 'g:Tree',
 '@value': [{'key': v[864],
             'value': {'@type': 'g:Tree',
                       '@value': [{'key': e[947][864-rdf:type->880],
                                   'value': {'@type': 'g:Tree',
                                             '@value': [{'key': v[880],
                                                         'value': {'@type': 'g:Tree',
                                                                   '@value': []}}]}},
                                  {'key': e[903][864-skos:inScheme->899],
                                   'value': {'@type': 'g:Tree',
                                             '@value': [{'key': v[899],
                                                         'value': {'@type': 'g:Tree',
                                                                   '@value': []}}]}},
                                  {'key': e[942][864-sh:path->940],
                                   'value': {'@type': 'g:Tree',
            